In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from selenium import webdriver
import time
import pandas as pd
from tqdm.notebook import tqdm
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
from fake_useragent import UserAgent

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
base_url = 'https://www.tripadvisor.com'
ua = UserAgent()
options = Options()
user_agent = ua.random
options.add_argument(f'user-agent={user_agent}')
options.add_argument("--incognito")


In [ ]:
pages = ['https://www.tripadvisor.com/Attractions-g293932-Activities-c20-t99-Yerevan.html','https://www.tripadvisor.com/Attractions-g293932-Activities-c20-t99-oa30-Yerevan.html' ]

In [ ]:
all_links = []
for page in tqdm(pages):
    headers = {'User-Agent': ua.random}
    content = requests.get(page, headers = headers, timeout = 3)
    soup = BeautifulSoup(content.text, 'html.parser')
    rest_container = soup.find('div', class_ = '_3LwzvfhX')
    btf_restcontainer = BeautifulSoup(str(rest_container), 'html.parser')
    
    narrowContainer = btf_restcontainer.find_all('div', class_ = '_6sUF3jUd')
    rawlink = [i.find('a', {'href': True}) for i in narrowContainer]
    link = [i['href'] for i in rawlink]
    full_link = [base_url + i for i in link]
    all_links.append(full_link)
all_links = flatten(all_links)
#all_links = set(all_links)

In [ ]:
#DONE
all_names = []
all_ratings = []
review_count = []
suggested_duration = []
all_address = []

for link in tqdm(all_links):
    browser = webdriver.Chrome(options=options)
    browser.get(link)
    time.sleep(2)
    source = browser.page_source
    soup = BeautifulSoup(source, 'html.parser')

    test_name = soup.find('h1', {'id':'HEADING'})
    if test_name:    
        name = soup.find('h1', {'id':'HEADING'}).text.strip()
        all_names.append(name)
    else:
        name = 'Not available'
        all_names.append(name)
    
    test = soup.find('span', {'class':'_2Hy7Xxdm'})
    if test:
        rating = soup.find('span', {'class':'_2Hy7Xxdm'}).text.strip()
        review = soup.find('span', {'class':'_3WF_jKL7 _1uXQPaAr'}).text.strip()
        all_ratings.append(rating)
        review_count.append(review)
    else:
        rating = 'No rating'
        review = '0 reviews'
        all_ratings.append(rating)
        review_count.append(review)
    
    
    test_duration = soup.find('div', {'class':'_2y7puPsQ'})
    if test_duration:    
        duration =  soup.find('div', {'class':'_2y7puPsQ'}).text.strip()
        suggested_duration.append(duration)
    else:
        duration = '2-3 hours'
        suggested_duration.append(duration)
            
    tester_longlat = soup.find('div', {'class':'LjCWTZdN'})
    if tester_longlat:
        address = soup.find('div', {'class':'LjCWTZdN'}).text.strip()
        all_address.append(address)
    else:
        address = 'Not available'
        all_address.append(address)


In [ ]:
open_hours = []
for link in tqdm(all_links):
    browser = webdriver.Chrome(options=options)
    browser.get(link)
    time.sleep(2)
    source = browser.page_source
    soup = BeautifulSoup(source, 'html.parser')
    test_hours = soup.find('div', {'class':'_2te4e9PG'})#.find_next('div').contents[0]
    print(test_hours)
    #<div class=""> See all hours</div>
    

None
None


KeyboardInterrupt: 

In [ ]:

    
    if test_hours:    
        hours =  soup.find('div', {'class':'_2y7puPsQ'}).text.strip()
        print(hours)
        
#######OR 

open_hours = []
for link in tqdm(all_links):
    browser = webdriver.Chrome(options=options)
    browser.get(link)
    time.sleep(2)
    source = browser.page_source
    soup = BeautifulSoup(source, 'html.parser')
     
    overview = soup.find('div', {'class':'_3UlLsY6_'})
    print(overview)
    now = overview.find('div', {'class':'_2GMWmZEQ'}).div.div.div
    test_hours = now.find_next_sibling('div')
    print(test_hours)
    
    #/* if test_hours:    
        #hours =  soup.find('div', {'class':'_2GMWmZEQ'}).text.strip()
        #print(hours)
        #suggested_duration.append(duration)*/

In [ ]:
df = pd.DataFrame({"Name" : all_names, "Rating" : all_ratings, "Review Count" : review_count, "Addresses" : all_address, "Suggested Duration":suggested_duration})
df.to_csv(r'C:\Users\Admin\Desktop\DataViz adoyan\Nightlife.csv', index = False)